In [2]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
import os
import seaborn as sns
import random 
import scipy as sc
# import babet as bb
import netCDF4
import dask

sns.set_theme(style="white")
sns.set_style("white")

random.seed(10)

In [4]:
dask.config.set(**{'array.slicing.split_large_chunks': True})

In [5]:
def accum2rate(ds):
    """
    Function to convert accumulated variables to conventional ones.
    Definition to convert accumulated variables to instantaneous.
    Written by Nick Leach.

    Input:
    ------

    Output:
    -------
    """
    # accumulated variables & scaling factors
    accumulated_vars = {'tp': 60 * 60 * 24 * 1e3,
                        'ttr': 1,
                        'tsr': 1,
                        'str': 1,
                        'ssr': 1,
                        'e': 1}
    accumulated_var_newunits = {'tp': 'mm day$^{-1}$',
                                'ttr': 'W m$^{-2}$',
                                'tsr': 'W m$^{-2}$',
                                'str': 'W m$^{-2}$',
                                'ssr': 'W m$^{-2}$',
                                'e':'m s$^{-1}$'}

    ds = ds.copy()
    oindex = ds.time
    inidate = pd.to_datetime(oindex[0].values)
    ds = ds.diff('time') / (ds.time.diff('time').astype(float) / 1e9 )
    ds = ds.reindex(time=oindex)
    return ds[1:]

def preproc_ds(ds):
    """
    Main pre-processing function
    Writtten by Nick Leach.

    Input:
    ------

    Output:
    -------
    """

    # accumulated variables & scaling factors
    accumulated_vars = {'tp': 60 * 60 * 24 * 1e3,
                        'ttr': 1,
                        'tsr': 1,
                        'str': 1,
                        'ssr': 1,
                        'e': 1}
    accumulated_var_newunits = {'tp': 'mm day$^{-1}$',
                                'ttr': 'W m$^{-2}$',
                                'tsr': 'W m$^{-2}$',
                                'str': 'W m$^{-2}$',
                                'ssr': 'W m$^{-2}$',
                                'e': 'm s$^{-1}$'}
    ds = ds.copy().squeeze()
    # set up aux data
    inidate = pd.to_datetime(ds.time[0].values)
    # expand dimensions to include extra info
    if not 'hDate' in ds:
        ds = ds.expand_dims({'inidate': [inidate]}).copy()

    if not 'number' in ds:
        ds = ds.expand_dims({'number': [0]}).copy()

    # put time dimension at front
    ds = ds.transpose('time', ...)
    ds = ds.copy(deep=True)

    # convert accumulated variables into instantaneous
    for var, sf in accumulated_vars.items():
        if var in ds.keys():
            ds[var].loc[dict(time=ds.time[1:])] = accum2rate(ds[var]) * sf
            # set first value to equal zero,
            # should be zero but isn't always
            ds[var].loc[dict(time=ds.time[0])] = 0
            ds[var].attrs['units'] = accumulated_var_newunits[var]
    return ds

# Import data

In [6]:
base_dir = '/gf5/predict/AWH019_ERMIS_ATMICP/Babet/DATA/MED-R/EXP/{}/EU025/sfc/pf'
file_dir = {'curr': base_dir.format('curr'),
            'pi': base_dir.format('pi'),
            'incr': base_dir.format('incr')}

In [7]:
xr.open_mfdataset('/gf3/predict2/AWH012_LEACH_NASTORM/DATA/MED-R/EXP/pi/EU025/sfc/pf/*.nc', preprocess=preproc_ds)

<xarray.Dataset>
Dimensions:    (time: 81, inidate: 3, longitude: 289, latitude: 163, number: 50)
Coordinates:
  * time       (time) datetime64[ns] 2022-02-10 ... 2022-02-20
  * inidate    (inidate) datetime64[ns] 2022-02-10 2022-02-14 2022-02-16
  * longitude  (longitude) float32 -27.0 -26.75 -26.5 -26.25 ... 44.5 44.75 45.0
  * latitude   (latitude) float32 73.5 73.25 73.0 72.75 ... 33.5 33.25 33.0
  * number     (number) int32 1 2 3 4 5 6 7 8 9 ... 42 43 44 45 46 47 48 49 50
Data variables: (12/14)
    sst        (time, inidate, number, latitude, longitude) float32 dask.array<chunksize=(14, 1, 50, 163, 289), meta=np.ndarray>
    fg10       (time, inidate, number, latitude, longitude) float32 dask.array<chunksize=(14, 1, 50, 163, 289), meta=np.ndarray>
    tcw        (time, inidate, number, latitude, longitude) float32 dask.array<chunksize=(14, 1, 50, 163, 289), meta=np.ndarray>
    tcwv       (time, inidate, number, latitude, longitude) float32 dask.array<chunksize=(14, 1, 50, 163, 289), meta=np.ndarray>
    msl        (time, inidate, number, latitude, longitude) float32 dask.array<chunksize=(14, 1, 50, 163, 289), meta=np.ndarray>
    tcc        (time, inidate, number, latitude, longitude) float32 dask.array<chunksize=(14, 1, 50, 163, 289), meta=np.ndarray>
    ...         ...
    u10        (time, inidate, number, latitude, longitude) float32 dask.array<chunksize=(14, 1, 50, 163, 289), meta=np.ndarray>
    v10        (time, inidate, number, latitude, longitude) float32 dask.array<chunksize=(14, 1, 50, 163, 289), meta=np.ndarray>
    u100       (time, inidate, number, latitude, longitude) float32 dask.array<chunksize=(14, 1, 50, 163, 289), meta=np.ndarray>
    v100       (time, inidate, number, latitude, longitude) float32 dask.array<chunksize=(14, 1, 50, 163, 289), meta=np.ndarray>
    mxcape6    (time, inidate, number, latitude, longitude) float32 dask.array<chunksize=(14, 1, 50, 163, 289), meta=np.ndarray>
    mxcapes6   (time, inidate, number, latitude, longitude) float32 dask.array<chunksize=(14, 1, 50, 163, 289), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2022-12-09 02:07:06 GMT by grib_to_netcdf-2.26.0: grib_to_n...

In [8]:
babet_curr = xr.open_mfdataset(os.path.join(file_dir['curr'], '*.nc'), preprocess=preproc_ds, engine='netcdf4')

# Maximum precipitation rate